In [ ]:
%load_ext autoreload
%autoreload 2
%load_ext line_profiler

In [ ]:
## Imports and environment variables 
import torch, wandb, os, pandas as pd 
from travis_attack.utils import set_seed, set_session_options, setup_logging, setup_parser, resume_wandb_run, display_all, print_important_cfg_vars
from travis_attack.config import Config
from travis_attack.models import prepare_models, get_optimizer
from travis_attack.data import ProcessedDataset
from travis_attack.trainer import Trainer
from travis_attack.insights import (postprocess_df, create_and_log_wandb_postrun_plots, get_training_dfs)
from fastcore.basics import in_jupyter

import logging 
logger = logging.getLogger("run")

import warnings
warnings.filterwarnings("ignore", message="Passing `max_length` to BeamSearchScorer is deprecated")  # works anyway for diverse beam search 

In [ ]:
from nbdev.export import notebook2script
notebook2script()

!jupyter nbconvert \
    --TagRemovePreprocessor.enabled=True \
    --TagRemovePreprocessor.remove_cell_tags="['hide']" \
    --TemplateExporter.exclude_markdown=True \
    --to python "run.ipynb"

Converted 00_utils.ipynb.
Converted 02_tests.ipynb.
Converted 03_config.ipynb.
Converted 07_models.ipynb.
Converted 10_data.ipynb.
Converted 20_trainer.ipynb.
Converted 25_insights.ipynb.
Converted baselines.ipynb.
Converted baselines_analysis.ipynb.
Converted index.ipynb.
Converted pp_eval_baselines.ipynb.
Converted run.ipynb.
Converted show_examples.ipynb.
Converted test_pp_model.ipynb.
[NbConvertApp] Converting notebook run.ipynb to python


In [ ]:
cfg = Config()  # default values
if not in_jupyter():  # override with any -- options when running with command line
    parser = setup_parser()
    newargs = vars(parser.parse_args())
    for k,v in newargs.items(): 
        if v is not None: 
            if k in cfg.pp.keys():  cfg.pp[k] = v
            else:                   setattr(cfg, k, v)
if cfg.use_small_ds:  cfg = cfg.small_ds()
set_seed(cfg.seed)
set_session_options()
setup_logging(cfg, disable_other_loggers=True)
vm_tokenizer, vm_model, pp_tokenizer, pp_model, ref_pp_model, sts_model, nli_tokenizer, nli_model, cfg = prepare_models(cfg)
optimizer = get_optimizer(cfg, pp_model)
ds = ProcessedDataset(cfg, vm_tokenizer, vm_model, pp_tokenizer, sts_model, load_processed_from_file=False)


travis_attack.data: INFO     Will load dataset simple with use_small_ds set to False
travis_attack.data: INFO     Will load dataset simple with use_small_ds set to False
travis_attack.data: INFO     Will load dataset simple with use_small_ds set to False


In [ ]:
cfg.wandb['mode'] = 'disabled'
trainer = Trainer(cfg, vm_tokenizer, vm_model, pp_tokenizer, pp_model, ref_pp_model, sts_model, nli_tokenizer, nli_model, optimizer,
                  ds, initial_eval=False)
#print_important_cfg_vars(cfg)
trainer.train()

travis_attack.trainer: INFO     Now on epoch 1 of 6
travis_attack.trainer: INFO     Now on epoch 1 of 6
travis_attack.trainer: INFO     Now on epoch 1 of 6


travis_attack.trainer: INFO     Now on epoch 2 of 6


travis_attack.trainer: INFO     Now on epoch 2 of 6
travis_attack.trainer: INFO     Now on epoch 2 of 6


travis_attack.trainer: INFO     Now on epoch 3 of 6
travis_attack.trainer: INFO     Now on epoch 3 of 6
travis_attack.trainer: INFO     Now on epoch 3 of 6


travis_attack.trainer: INFO     Now on epoch 4 of 6
travis_attack.trainer: INFO     Now on epoch 4 of 6
travis_attack.trainer: INFO     Now on epoch 4 of 6


travis_attack.trainer: INFO     Now on epoch 5 of 6
travis_attack.trainer: INFO     Now on epoch 5 of 6
travis_attack.trainer: INFO     Now on epoch 5 of 6


travis_attack.trainer: INFO     Now on epoch 6 of 6
travis_attack.trainer: INFO     Now on epoch 6 of 6
travis_attack.trainer: INFO     Now on epoch 6 of 6


In [ ]:
df = pd.read_csv(f'{cfg.path_run}training_step.csv')
#display_all(df.query('idx==1'))
df.columns

Index(['idx', 'epoch', 'orig', 'pp', 'orig_truelabel_probs',
       'pp_truelabel_probs', 'pp_predclass_probs', 'label', 'pp_predclass',
       'label_flip', 'vm_scores', 'sts_scores', 'pp_letter_diff',
       'pp_letter_percent', 'contradiction_scores', 'reward', 'pp_logp',
       'ref_logp', 'kl_div', 'reward_penalty', 'reward_with_penalty', 'loss',
       'batch_num', 'global_step', 'acc_num', 'loss_sum', 'loss_batch',
       'label_flip_fraction', 'orig_length', 'orig_batch_size', 'pp_length',
       'pp_batch_size', 'time_generate_pp', 'time_loss_fn', 'time_reward_fn',
       'time_vm_scores', 'time_sts_scores', 'time_pp_letter_diff',
       'time_contradiction_scores', 'time_pp_logp', 'time_log_entropy',
       'time_log_token_probabilities', 'time_ref_logprobs',
       'time_loss_fn_loss_calc', 'time_backwards', 'time_calc_gradient_norm',
       'time_opt_step_and_calc_param_norm', 'time_opt_step'],
      dtype='object')

In [ ]:
df = pd.read_csv(f'{cfg.path_run}train.csv')
#display_all(df.query('idx==1'))
df.columns

Index(['idx', 'epoch', 'orig', 'pp', 'pp_idx', 'is_adv_example', 'is_valid_pp',
       'label_flip', 'reward', 'vm_scores', 'sts_scores',
       'contradiction_scores', 'pp_letter_diff', 'label',
       'orig_truelabel_probs', 'pp_truelabel_probs', 'pp_predclass',
       'pp_predclass_probs', 'orig_n_letters', 'pp_letter_percent'],
      dtype='object')

In [ ]:
df

,idx,epoch,orig,pp,pp_idx,is_adv_example,is_valid_pp,label_flip,reward,vm_scores,sts_scores,contradiction_scores,pp_letter_diff,label,orig_truelabel_probs,pp_truelabel_probs,pp_predclass,pp_predclass_probs,orig_n_letters,pp_letter_percent
0,1,1,I do not like this movie,I don't like this movie.,orig_1-epoch_1-pp_1,0,1,0,0.000000,-0.031588,0.870033,0.003474,0,0,0.892648,0.924236,0,0.924236,24,1.000000
1,1,1,I do not like this movie,I don't like this movie.,orig_1-epoch_1-pp_2,0,1,0,0.000000,-0.031588,0.870033,0.003474,0,0,0.892648,0.924236,0,0.924236,24,1.000000
2,1,1,I do not like this movie,I don't like this movie '',orig_1-epoch_1-pp_3,0,1,0,0.471678,0.039307,0.876496,0.003023,-2,0,0.892648,0.853342,0,0.853342,24,1.083333
3,1,1,I do not like this movie,It's not a good movie -,orig_1-epoch_1-pp_4,0,1,0,0.000000,-0.038755,0.754484,0.003561,1,0,0.892648,0.931403,0,0.931403,24,0.958333
4,1,1,I do not like this movie,It's not a good movie,orig_1-epoch_1-pp_5,0,1,0,0.000000,-0.032068,0.755407,0.003221,3,0,0.892648,0.924717,0,0.924717,24,0.875000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,3,6,I hate this apple,Why do I hate this apple?,orig_3-epoch_6-pp_4,0,1,0,0.000000,-0.012583,0.943637,0.034731,-8,0,0.838063,0.850647,0,0.850647,17,1.470588
188,3,6,I hate this apple,i hate this apple.,orig_3-epoch_6-pp_5,0,1,0,0.000000,-0.045688,0.941781,0.004530,-1,0,0.838063,0.883751,0,0.883751,17,1.058824
189,3,6,I hate this apple,It's a bad apple.,orig_3-epoch_6-pp_6,0,1,0,0.000000,-0.089815,0.750743,0.004766,0,0,0.838063,0.927879,0,0.927879,17,1.000000
190,3,6,I hate this apple,Why do I hate that apple?,orig_3-epoch_6-pp_7,0,1,0,0.031647,0.002637,0.915063,0.033142,-8,0,0.838063,0.835426,0,0.835426,17,1.470588


In [ ]:
trainer.run.finish()

In [ ]:
# ## TO RESUME RUN
cfg = Config()
cfg.run_id = 'b9r88lmz'
cfg.run_name = "fresh-cosmos-151"
cfg.path_run = f"{cfg.path_checkpoints}{cfg.run_name}/"
run = resume_wandb_run(cfg)


In [ ]:
df_d = get_training_dfs(cfg.path_run, postprocessed=False)
for k, df in df_d.items(): 
    df_d[k] = postprocess_df(df, filter_idx=None, num_proc=1)
    df_d[k].to_pickle(f"{cfg.path_run}{k}_postprocessed.pkl")    
create_and_log_wandb_postrun_plots(df_d)
trainer.run.finish()
#run.finish()

travis_attack.insights: INFO     Dataframes have shapes ['training_step: (2000, 48)', 'train: (1600, 20)', 'valid: (1600, 20)']


KeyError: 'sts_scores'